# Semantic Kernel Process Framework and Human-in-the-Loop

This notebook introduces two powerful concepts for building production-ready AI systems:

1. **Semantic Kernel Process Framework**: Event-driven business process orchestration
2. **Human-in-the-Loop (HITL)**: Patterns for human oversight and approval in AI workflows

## What is the Process Framework?

The Semantic Kernel Process Framework enables you to build complex, event-driven business processes where each step can:
- Invoke AI agents
- Execute native code
- Emit events to trigger subsequent steps
- Branch based on conditions
- Wait for human approval (HITL)

Think of it as a **workflow engine** specifically designed for AI-powered applications.

### Key Concepts

```
┌─────────────────────────────────────────────────────────┐
│                    Process                               │
│  ┌──────────┐      ┌──────────┐      ┌──────────┐      │
│  │  Step 1  │─────▶│  Step 2  │─────▶│  Step 3  │      │
│  │ Translate│      │ Summarize│      │  Review  │      │
│  └────┬─────┘      └────┬─────┘      └────┬─────┘      │
│       │ Event           │ Event           │ Event      │
│       │ (Translated)    │ (Summarized)    │ (Approved) │
│       ▼                 ▼                 ▼            │
└─────────────────────────────────────────────────────────┘
```

### Benefits

1. **Event-Driven Architecture**: Loose coupling between steps
2. **Observability**: Built-in support for tracing and monitoring
3. **Reusability**: Steps can be reused across processes
4. **Testability**: Each step can be tested independently
5. **Scalability**: Steps can run in separate services

## What is Human-in-the-Loop (HITL)?

Human-in-the-Loop is a design pattern where human oversight is integrated into AI workflows for:
- **Quality Assurance**: Review AI outputs before they're used
- **Compliance**: Ensure regulatory requirements are met
- **Safety**: Prevent harmful or incorrect AI actions
- **Learning**: Gather human feedback to improve AI models

### HITL Patterns

1. **Approval Gate**: Human must approve before proceeding
2. **Review & Edit**: Human can modify AI output
3. **Escalation**: Complex cases routed to humans
4. **Feedback Loop**: Human corrections train the model

## Integration with .NET Aspire

The Process Framework integrates beautifully with .NET Aspire for:
- **Distributed Tracing**: Track process execution across services
- **Service Discovery**: Automatically connect to agent services
- **Monitoring**: View metrics and logs in Aspire dashboard
- **Development**: Hot reload and restart individual services

## References

- [Semantic Kernel Process Framework Documentation](https://learn.microsoft.com/semantic-kernel/overview)
- [Process Framework with Aspire Sample](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/Demos/ProcessFrameworkWithAspire)
- [.NET Aspire Overview](https://learn.microsoft.com/dotnet/aspire/get-started/aspire-overview)

## Setup and Configuration

In [19]:
// Load configuration settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

Console.WriteLine($"Configuration loaded:");
Console.WriteLine($"- Model: {model}");
Console.WriteLine($"- Endpoint: {azureEndpoint}");
Console.WriteLine($"- API Key configured: {!string.IsNullOrEmpty(apiKey)}");

Configuration loaded:
- Model: gpt-4o
- Endpoint: https://eastus.api.cognitive.microsoft.com/
- API Key configured: True


## Install Required Packages

The Process Framework is currently in preview, so we need to enable experimental features.

In [ ]:
// Install Semantic Kernel packages

#r "nuget: Microsoft.SemanticKernel, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Process.Abstractions, 1.67.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.67.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.67.1-alpha"



using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;  // Alias to avoid conflicts
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Process;
using System.ComponentModel;



// Suppress experimental API warnings for Process Framework
#pragma warning disable SKEXP0080
#pragma warning disable SKEXP0001



Console.WriteLine("Packages loaded successfully.");

Installed Packages Microsoft.SemanticKernel, 1.67.1 Microsoft.SemanticKernel.Agents.Core, 1.67.1 Microsoft.SemanticKernel.Process.Abstractions, 1.67.1-alpha Microsoft.SemanticKernel.Process.Core, 1.67.1-alpha Microsoft.SemanticKernel.Process.LocalRuntime, 1.67.1-alpha

Packages loaded successfully.


In [21]:
Console.WriteLine("C# language version set to 13");

C# language version set to 13


## Part 1: Understanding Process Framework Basics

Let's start with a simple process to understand the core concepts.

### Scenario: Document Processing Pipeline

We'll build a process that:
1. Receives a document
2. Translates it to English
3. Summarizes the translation
4. Extracts key points

```
Input Document
     │
     ▼
[Translate] ─TranslatedEvent─▶ [Summarize] ─SummarizedEvent─▶ [Extract Key Points]
     │                              │                                  │
     └──────────────────────────────┴──────────────────────────────────┘
                            Event-Driven Flow
```

### Step 1: Define Process Events

Events are the communication mechanism between steps.

In [22]:
// Define process events as constants
public static class ProcessEvents
{
    // Input events
    public const string StartProcess = nameof(StartProcess);
    
    // Step completion events
    public const string DocumentTranslated = nameof(DocumentTranslated);
    public const string DocumentSummarized = nameof(DocumentSummarized);
    public const string KeyPointsExtracted = nameof(KeyPointsExtracted);
    
    // HITL events
    public const string HumanApprovalRequired = nameof(HumanApprovalRequired);
    public const string HumanApproved = nameof(HumanApproved);
    public const string HumanRejected = nameof(HumanRejected);
}

Console.WriteLine("Process events defined.");

Process events defined.


### Step 2: Create Kernel for Process

The kernel provides AI services to our process steps.

In [23]:
// Create kernel with AI services
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
Kernel kernel = builder.Build();

Console.WriteLine("Kernel created for process framework.");

Kernel created for process framework.


### Step 3: Define Process Steps

Each step is a class that inherits from `KernelProcessStep` and defines functions that can be invoked.

In [24]:
#pragma warning disable SKEXP0080
// Translation Step
public class TranslateStep : KernelProcessStep
{
    public static class Functions
    {
        public const string Translate = nameof(Translate);
    }

    [KernelFunction(Functions.Translate)]
    public async Task TranslateAsync(
        KernelProcessStepContext context,
        Kernel kernel,
        string textToTranslate)
    {
        Console.WriteLine($"\n[TranslateStep] Translating text...");
        
        var chatService = kernel.GetRequiredService<IChatCompletionService>();
        
        var prompt = $"Translate the following text to English. If already in English, return as-is:\n\n{textToTranslate}";
        var result = await chatService.GetChatMessageContentAsync(prompt);
        
        var translatedText = result.Content;
        Console.WriteLine($"[TranslateStep] Translation completed: {translatedText?.Substring(0, Math.Min(100, translatedText?.Length ?? 0))}...");
        
        // Emit event to trigger next step
        await context.EmitEventAsync(new KernelProcessEvent 
        { 
            Id = ProcessEvents.DocumentTranslated, 
            Data = translatedText 
        });
    }
}

// Summarization Step
public class SummarizeStep : KernelProcessStep
{
    public static class Functions
    {
        public const string Summarize = nameof(Summarize);
    }

    [KernelFunction(Functions.Summarize)]
    public async Task SummarizeAsync(
        KernelProcessStepContext context,
        Kernel kernel,
        string textToSummarize)
    {
        Console.WriteLine($"\n[SummarizeStep] Summarizing text...");
        
        var chatService = kernel.GetRequiredService<IChatCompletionService>();
        
        var prompt = $"Provide a concise summary of the following text in 2-3 sentences:\n\n{textToSummarize}";
        var result = await chatService.GetChatMessageContentAsync(prompt);
        
        var summary = result.Content;
        Console.WriteLine($"[SummarizeStep] Summary: {summary}");
        
        // Emit event to trigger next step
        await context.EmitEventAsync(new KernelProcessEvent 
        { 
            Id = ProcessEvents.DocumentSummarized, 
            Data = summary 
        });
    }
}

// Key Points Extraction Step
public class ExtractKeyPointsStep : KernelProcessStep
{
    public static class Functions
    {
        public const string Extract = nameof(Extract);
    }

    [KernelFunction(Functions.Extract)]
    public async Task ExtractAsync(
        KernelProcessStepContext context,
        Kernel kernel,
        string textToAnalyze)
    {
        Console.WriteLine($"\n[ExtractKeyPointsStep] Extracting key points...");
        
        var chatService = kernel.GetRequiredService<IChatCompletionService>();
        
        var prompt = $"Extract 3-5 key points from this text as a bulleted list:\n\n{textToAnalyze}";
        var result = await chatService.GetChatMessageContentAsync(prompt);
        
        var keyPoints = result.Content;
        Console.WriteLine($"[ExtractKeyPointsStep] Key Points:\n{keyPoints}");
        
        // Emit completion event
        await context.EmitEventAsync(new KernelProcessEvent 
        { 
            Id = ProcessEvents.KeyPointsExtracted, 
            Data = keyPoints 
        });
    }
}

Console.WriteLine("Process steps defined:");
Console.WriteLine("- TranslateStep");
Console.WriteLine("- SummarizeStep");
Console.WriteLine("- ExtractKeyPointsStep");

Process steps defined:
- TranslateStep
- SummarizeStep
- ExtractKeyPointsStep


### Step 4: Build and Run the Process

Now let's wire up the steps and run the process.

In [28]:
#pragma warning disable SKEXP0080

// Sample document to process (in Italian)
var sampleDocument = @"
L'intelligenza artificiale sta trasformando il modo in cui lavoriamo e viviamo. 
Le aziende stanno adottando l'IA per automatizzare processi, migliorare il servizio 
clienti e prendere decisioni basate sui dati. Tuttavia, è importante considerare 
le implicazioni etiche e garantire che l'IA sia utilizzata in modo responsabile.
";

Console.WriteLine("Sample Document (Italian):");
Console.WriteLine("=" + new string('=', 50));
Console.WriteLine(sampleDocument);
Console.WriteLine();

// Build the process
var processBuilder = new ProcessBuilder("DocumentProcessingPipeline");

// Add steps to the process
var translateStep = processBuilder.AddStepFromType<TranslateStep>();
var summarizeStep = processBuilder.AddStepFromType<SummarizeStep>();
var extractStep = processBuilder.AddStepFromType<ExtractKeyPointsStep>();

// Wire up the event flow
// When process starts, send to translate step
processBuilder
    .OnInputEvent(ProcessEvents.StartProcess)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        translateStep, 
        TranslateStep.Functions.Translate, 
        parameterName: "textToTranslate"));

// When translation completes, send to summarize step
translateStep
    .OnEvent(ProcessEvents.DocumentTranslated)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        summarizeStep, 
        SummarizeStep.Functions.Summarize, 
        parameterName: "textToSummarize"));

// When summarization completes, send to extract key points step
summarizeStep
    .OnEvent(ProcessEvents.DocumentSummarized)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        extractStep, 
        ExtractKeyPointsStep.Functions.Extract, 
        parameterName: "textToAnalyze"));

// When key points are extracted, stop the process
extractStep
    .OnEvent(ProcessEvents.KeyPointsExtracted)
    .StopProcess();

// Build and start the process
var process = processBuilder.Build();

Console.WriteLine("\n" + "=" + new string('=', 50));
Console.WriteLine("Starting Process Execution");
Console.WriteLine("=" + new string('=', 50));

var runningProcess = await process.StartAsync(
    kernel,
    new KernelProcessEvent 
    { 
        Id = ProcessEvents.StartProcess, 
        Data = sampleDocument 
    }
);

Console.WriteLine("\n" + "=" + new string('=', 50));
Console.WriteLine("Process completed successfully!");
Console.WriteLine("=" + new string('=', 50));

Sample Document (Italian):

L'intelligenza artificiale sta trasformando il modo in cui lavoriamo e viviamo. 
Le aziende stanno adottando l'IA per automatizzare processi, migliorare il servizio 
clienti e prendere decisioni basate sui dati. Tuttavia, è importante considerare 
le implicazioni etiche e garantire che l'IA sia utilizzata in modo responsabile.



Starting Process Execution

[TranslateStep] Translating text...
[TranslateStep] Translation completed: Artificial intelligence is transforming the way we work and live. Companies are adopting AI to autom...

[SummarizeStep] Summarizing text...
[SummarizeStep] Summary: Artificial intelligence is revolutionizing work and daily life by automating tasks, enhancing customer service, and enabling data-driven decisions. While its adoption grows, ethical considerations and responsible use remain essential.

[ExtractKeyPointsStep] Extracting key points...
[ExtractKeyPointsStep] Key Points:
- Artificial intelligence is transforming work and d

## Part 2: Human-in-the-Loop (HITL) Patterns

Now let's add human oversight to our process. We'll implement an approval gate where a human must review and approve the summary before key points are extracted.

### HITL Architecture

```
[Translate] ─▶ [Summarize] ─▶ [Human Review] ─▶ [Extract Key Points]
                                    │
                                    ├─ Approved ──▶ Continue
                                    │
                                    └─ Rejected ──▶ Re-summarize
```

### Implementing Human Review Step

This step simulates human review. In a real application, this would:
- Store the data in a queue or database
- Notify a human reviewer (email, UI notification, etc.)
- Wait for human input
- Resume process based on human decision

In [29]:
#pragma warning disable SKEXP0080
// Human Review Step with approval logic
public class HumanReviewStep : KernelProcessStep
{
    public static class Functions
    {
        public const string Review = nameof(Review);
    }

    [KernelFunction(Functions.Review)]
    public async Task ReviewAsync(
        KernelProcessStepContext context,
        string contentToReview)
    {
        Console.WriteLine($"\n[HumanReviewStep] HUMAN REVIEW REQUIRED");
        Console.WriteLine($"=" + new string('=', 50));
        Console.WriteLine($"Content for Review:");
        Console.WriteLine(contentToReview);
        Console.WriteLine($"=" + new string('=', 50));
        
        // Simulate human review
        // In production, this would:
        // 1. Store content in a review queue
        // 2. Notify reviewer (email, Slack, Teams, etc.)
        // 3. Wait for async approval via API or UI
        // 4. Resume process when human responds
        
        Console.WriteLine($"\n[Simulation] Waiting for human approval...");
        await Task.Delay(1000); // Simulate review time
        
        // Simulate approval logic
        // In reality, this would come from external input
        bool isApproved = true; // Change to false to test rejection
        
        if (isApproved)
        {
            Console.WriteLine($"[HumanReviewStep] ✓ APPROVED by reviewer");
            await context.EmitEventAsync(new KernelProcessEvent 
            { 
                Id = ProcessEvents.HumanApproved, 
                Data = contentToReview 
            });
        }
        else
        {
            Console.WriteLine($"[HumanReviewStep] ✗ REJECTED by reviewer");
            await context.EmitEventAsync(new KernelProcessEvent 
            { 
                Id = ProcessEvents.HumanRejected, 
                Data = "Needs more detail" 
            });
        }
    }
}

Console.WriteLine("Human Review Step defined.");

Human Review Step defined.


### Building Process with HITL

In [32]:
#pragma warning disable SKEXP0080

// Build process with Human-in-the-Loop
var hitlProcessBuilder = new ProcessBuilder("DocumentProcessingWithHITL");

// Add steps
var translateStep2 = hitlProcessBuilder.AddStepFromType<TranslateStep>();
var summarizeStep2 = hitlProcessBuilder.AddStepFromType<SummarizeStep>();
var reviewStep = hitlProcessBuilder.AddStepFromType<HumanReviewStep>();
var extractStep2 = hitlProcessBuilder.AddStepFromType<ExtractKeyPointsStep>();

// Wire up event flow with HITL
hitlProcessBuilder
    .OnInputEvent(ProcessEvents.StartProcess)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        translateStep2, 
        TranslateStep.Functions.Translate, 
        parameterName: "textToTranslate"));

translateStep2
    .OnEvent(ProcessEvents.DocumentTranslated)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        summarizeStep2, 
        SummarizeStep.Functions.Summarize, 
        parameterName: "textToSummarize"));

// After summarization, require human review
summarizeStep2
    .OnEvent(ProcessEvents.DocumentSummarized)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        reviewStep, 
        HumanReviewStep.Functions.Review, 
        parameterName: "contentToReview"));

// If approved, continue to extract key points
reviewStep
    .OnEvent(ProcessEvents.HumanApproved)
    .SendEventTo(new ProcessFunctionTargetBuilder(
        extractStep2, 
        ExtractKeyPointsStep.Functions.Extract, 
        parameterName: "textToAnalyze"));

// If rejected, could re-route to summarization with feedback
// (Not implemented in this example, but shows the pattern)
reviewStep
    .OnEvent(ProcessEvents.HumanRejected)
    .StopProcess(); // In production, would loop back or escalate

extractStep2
    .OnEvent(ProcessEvents.KeyPointsExtracted)
    .StopProcess();

// Build and run
var hitlProcess = hitlProcessBuilder.Build();

Console.WriteLine("\n" + "=" + new string('=', 50));
Console.WriteLine("Starting Process with HITL");
Console.WriteLine("=" + new string('=', 50));

var runningHitlProcess = await hitlProcess.StartAsync(
    kernel,
    new KernelProcessEvent 
    { 
        Id = ProcessEvents.StartProcess, 
        Data = sampleDocument 
    }
);

Console.WriteLine("\n" + "=" + new string('=', 50));
Console.WriteLine("HITL Process completed!");
Console.WriteLine("=" + new string('=', 50));


Starting Process with HITL

[TranslateStep] Translating text...
[TranslateStep] Translation completed: Artificial intelligence is transforming the way we work and live.  
Companies are adopting AI to aut...

[SummarizeStep] Summarizing text...
[SummarizeStep] Summary: Artificial intelligence is revolutionizing work and daily life by automating processes, enhancing customer service, and enabling data-driven decisions. While its adoption grows, it is crucial to address ethical concerns and promote responsible usage.

[HumanReviewStep] HUMAN REVIEW REQUIRED
Content for Review:
Artificial intelligence is revolutionizing work and daily life by automating processes, enhancing customer service, and enabling data-driven decisions. While its adoption grows, it is crucial to address ethical concerns and promote responsible usage.

[Simulation] Waiting for human approval...
[HumanReviewStep] ✓ APPROVED by reviewer

[ExtractKeyPointsStep] Extracting key points...
[ExtractKeyPointsStep] Key Points

## Real-World HITL Implementation Patterns

### Pattern 1: Queue-Based Review

```csharp
// Store review request in database/queue
public async Task RequestReviewAsync(string content)
{
    var reviewRequest = new ReviewRequest
    {
        Id = Guid.NewGuid(),
        Content = content,
        Status = ReviewStatus.Pending,
        CreatedAt = DateTime.UtcNow
    };
    
    await _reviewQueue.EnqueueAsync(reviewRequest);
    await _notificationService.NotifyReviewersAsync(reviewRequest);
    
    // Process will pause here
    // Resume via webhook when reviewer responds
}
```

### Pattern 2: Webhook Resume

```csharp
// API endpoint for reviewer to respond
[HttpPost("api/review/{reviewId}/respond")]
public async Task<IActionResult> RespondToReview(
    Guid reviewId, 
    ReviewResponse response)
{
    // Resume the process with human decision
    await _processRuntime.SendEventAsync(
        response.Approved 
            ? ProcessEvents.HumanApproved 
            : ProcessEvents.HumanRejected,
        response.Feedback
    );
    
    return Ok();
}
```

### Pattern 3: Timeout & Escalation

```csharp
public class HumanReviewStepWithTimeout : KernelProcessStep
{
    [KernelFunction]
    public async Task ReviewWithTimeoutAsync(
        KernelProcessStepContext context,
        string content)
    {
        var reviewTask = RequestReviewAsync(content);
        var timeoutTask = Task.Delay(TimeSpan.FromHours(24));
        
        var completedTask = await Task.WhenAny(reviewTask, timeoutTask);
        
        if (completedTask == timeoutTask)
        {
            // Escalate to manager or auto-approve
            await context.EmitEventAsync(
                new KernelProcessEvent 
                { 
                    Id = "ReviewTimeout", 
                    Data = content 
                });
        }
    }
}
```

## Integration with .NET Aspire

The Process Framework shines when combined with .NET Aspire:

### Example: Distributed Process with Aspire

```csharp
// AppHost Program.cs
var builder = DistributedApplication.CreateBuilder(args);

// Define services for each agent/step
var translatorAgent = builder.AddProject<TranslatorAgent>("translator")
    .WithExternalHttpEndpoints();

var summaryAgent = builder.AddProject<SummaryAgent>("summary")
    .WithExternalHttpEndpoints();

var reviewService = builder.AddProject<ReviewService>("review")
    .WithExternalHttpEndpoints();

// Process orchestrator that coordinates all steps
var orchestrator = builder.AddProject<ProcessOrchestrator>("orchestrator")
    .WithReference(translatorAgent)
    .WithReference(summaryAgent)
    .WithReference(reviewService);

builder.Build().Run();
```

### Benefits:

1. **Service Discovery**: Automatic connection between services
2. **Distributed Tracing**: See process flow across services in Aspire dashboard
3. **Independent Scaling**: Scale each agent separately
4. **Hot Reload**: Restart individual agents without stopping process
5. **Monitoring**: View metrics, logs, and traces in real-time

## Key Takeaways

### Process Framework

1. **Event-Driven**: Steps communicate via events, enabling loose coupling
2. **Declarative**: Define process structure, framework handles execution
3. **Observable**: Built-in support for tracing and monitoring
4. **Composable**: Reuse steps across multiple processes
5. **Scalable**: Distribute steps across services

### Human-in-the-Loop

1. **Safety**: Prevent harmful AI actions with human oversight
2. **Quality**: Ensure outputs meet business standards
3. **Compliance**: Satisfy regulatory requirements
4. **Learning**: Gather feedback to improve AI models
5. **Trust**: Build confidence in AI systems

### Best Practices

1. **Clear Events**: Use descriptive event names
2. **Idempotent Steps**: Steps should handle duplicate events
3. **Error Handling**: Implement retry and fallback logic
4. **Timeout Management**: Don't wait forever for humans
5. **Audit Trail**: Log all HITL decisions

## Next Steps

- Explore the [Process Framework with Aspire sample](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/Demos/ProcessFrameworkWithAspire)
- Read [.NET Aspire documentation](https://learn.microsoft.com/dotnet/aspire/get-started/aspire-overview)
- Try Assignment 5 for hands-on practice with Process Framework and HITL
- Experiment with complex branching and parallel execution patterns